In [3]:
import requests, re, pandas as pd
from bs4 import BeautifulSoup

URL = "https://www.wivern.com/security20170414.html"
r = requests.get(URL, timeout=20); r.raise_for_status()
soup = BeautifulSoup(r.text, "html.parser")
text = soup.get_text("\n")

lines = text.splitlines()
ip_re = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
results = []
for line in lines:
    m = ip_re.search(line)
    if m:
        ip = m.group(0)
        ts_match = re.search(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', line)
        ts = ts_match.group(0) if ts_match else ""
        results.append((ip, ts))

dedup = {}
for ip, ts in results:
    if ip not in dedup or (ts and ts > dedup[ip]):
        dedup[ip] = ts

df = pd.DataFrame(list(dedup.items()), columns=["ip","timestamp"])
df.to_csv("tor_exits.csv", index=False, encoding="utf-8")
df


,ip,timestamp
0,171.25.193.25,2025-09-29 22:16:48
1,80.67.167.81,2025-09-29 22:46:24
2,198.98.51.189,2025-09-29 22:21:14
3,89.58.26.216,2025-09-29 22:22:13
4,109.70.100.4,2025-09-29 22:43:04
...,...,...
1131,94.16.115.121,2025-09-29 22:36:52
1132,193.239.232.235,2025-09-29 22:10:37
1133,151.243.109.167,2025-09-29 22:18:25
1134,37.228.129.162,2025-09-29 19:02:47
